In [2]:
import hmmlearn
import numpy as np

In [3]:
import pickle 

def save_obj(obj, name):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [4]:
syllables_dict = load_obj("syllables_dict_with_punc")

In [6]:
print(syllables_dict)
print(sorted(np.unique(sentences)))
print(len(syllables.keys()))

{"'gainst": {'index': 0, 'syllabs': [1]}, "'greeing": {'index': 1, 'end_syllab': 1, 'syllabs': [2]}, "'scaped": {'index': 2, 'syllabs': [1]}, "'tis": {'index': 3, 'syllabs': [1]}, "'twixt": {'index': 4, 'syllabs': [1]}, 'a': {'index': 5, 'syllabs': [1]}, 'a-doting': {'index': 6, 'end_syllab': 2, 'syllabs': [3]}, 'abhor': {'index': 7, 'syllabs': [2]}, 'abide': {'index': 8, 'syllabs': [2]}, 'able': {'index': 9, 'syllabs': [2]}, 'about': {'index': 10, 'syllabs': [2]}, 'above': {'index': 11, 'syllabs': [2]}, 'absence': {'index': 12, 'syllabs': [2]}, 'absent': {'index': 13, 'syllabs': [2]}, 'abundance': {'index': 14, 'syllabs': [3]}, 'abundant': {'index': 15, 'syllabs': [3]}, 'abuse': {'index': 16, 'syllabs': [2]}, 'abused': {'index': 17, 'syllabs': [2]}, 'abuses': {'index': 18, 'syllabs': [3]}, 'abysm': {'index': 19, 'syllabs': [2]}, 'accents': {'index': 20, 'syllabs': [2]}, 'acceptable': {'index': 21, 'syllabs': [4]}, 'acceptance': {'index': 22, 'syllabs': [3]}, 'accessary': {'index': 23,

NameError: name 'sentences' is not defined

In [7]:
A = load_obj("trans_mat_epoch11")
O = load_obj("emiss_mat_epoch11")

emission_size = O.shape[1]
latent_size = A.shape[0]
print(A.shape, O.shape)
# latent_size = 10
# emission_size = 5
# ["I", "want", "orange", "door", "really"]
# A = np.random.random([latent_size, latent_size])
# O = np.random.random([latent_size, emission_size])
# A = A/np.tile(np.expand_dims(np.sum(A, axis = 1), axis = 1), latent_size)
# O = O/np.tile(np.expand_dims(np.sum(O, axis = 1), axis = 1), emission_size)
# O = O/np.tile(np.expand_dims(np.sum(O, axis = 1), axis = 1), latent_size)

(16, 16) (16, 3214)


In [8]:
print(list(syllables_dict.keys())[2])
print(syllables_dict[list(syllables_dict.keys())[2]])

'scaped
{'index': 2, 'syllabs': [1]}


## Generate Regular Poem

In [45]:
# GENERATE SIMPLE SONNET NO RHYME SCHEME 
import random

all_emissions = list(range(emission_size))
all_states = list(range(latent_size))
emission_words = list(syllables_dict.keys())
max_syllab = 10
poem = []
poem_syllabs = []
for i in range(14):
    syllable_count = 0
    t = 0
    states = [0]*(max_syllab+10)
    lines = []
    line_syllabs = []
    while syllable_count < max_syllab:
        if syllable_count == 0:
            states[t] = random.choices(all_states, weights = [1/latent_size]*latent_size)
            idx = random.choices(all_emissions, weights = O[states[t]][0])[0]
            word = emission_words[idx]
            
        else:
            states[t] = random.choices(all_states, weights = A[states[t-1]][0])
            idx = random.choices(all_emissions, weights = O[states[t-1]][0])[0]
            word = emission_words[idx]
#             print(states, idx, word)
        
        # update syllable count
        word_syllables = syllables_dict[word]["syllabs"]
        syllab = random.choices(word_syllables, weights = [1/len(word_syllables)]*len(word_syllables))[0]
         
        # if exceeds syllable count, try again
        if (syllable_count + syllab) > max_syllab:
            while (syllable_count + syllab) > max_syllab:
                states[t] = random.choices(all_states, weights = A[states[t-1]][0])
                idx = random.choices(all_emissions, weights = O[states[t-1]][0])[0]
                word = emission_words[idx]
                word_syllables = syllables_dict[word]["syllabs"]
                syllab = random.choices(word_syllables, weights = [1/len(word_syllables)]*len(word_syllables))[0]
        syllable_count += random.choices(word_syllables, weights = [1/len(word_syllables)]*len(word_syllables))[0]
        lines.append(word)
        line_syllabs.append(syllab)
        t += 1
    poem.append(lines)
    poem_syllabs.append(line_syllabs)
    
sonnet = ""
for i in range(len(poem)):
    line = poem[i]
    for ind in range(len(poem[i])-1, -1, -1):
        sonnet += str(line[ind])+ " "
    sonnet += "\n"
    
print(sonnet)
# print(np.array(poem))
# print([len(poem[i]) for i in range(len(poem))])
# print([len(poem_syllabs[i]) for i in range(len(poem_syllabs))])
# print([sum(poem_syllabs[i]) for i in range(len(poem_syllabs))])
# print(np.array(poem_syllabs))

in love ruin , a your if on did soil , 
born thy thy i which she the i ever 
curious read do large how drained to thee doth 
thy works my so your arise windy ) when , 
do for , love and makeless feeding much me . : : 
the it blushing to as if succeeding , 
being where the to long read am i , : spent . 
recounting my more , controlling kindness 
love conceit so be possession behind 
or crown are being effectually which . 
not bearing spite thy thyself you thee your 
should dost with , spends heart in so by that that 
old the alone bid they and wilt it in 
gold then , thoughts such the time blessed-fair friends , , 



In [41]:
sentences_idx = load_obj("sentences_idx_arr_with_punc_all")
sentences = load_obj("sentences_arr_with_punc_all")
syllables = load_obj("syllables_dict_with_punc")

## Generate Rhyming Poem

In [42]:
rhyme_scheme = load_obj("rhyming_dict")
end_words = load_obj("end_words_and_pronunciations")

In [43]:
# Select rhyming words 

np.random.shuffle(end_words)
rhyme_pairs = []

atoms_found = 0
ind = 0
while atoms_found < 7:
    w,l,p = end_words[ind]
    if len(p) > 2:
        pattern = p[-2:]
    else:
        pattern = p[-1]
    
    for key in rhyme_scheme.keys():
        words = rhyme_scheme[key]["words"]
        rhyme = rhyme_scheme[key]["rhyme scheme"] 
        if rhyme == pattern and (len(words) > 1):
            indices = list(range(len(words)))
            shuffled = np.random.permutation(indices)
            rhyme_pairs.append([words[shuffled[0]], words[shuffled[1]]])
            atoms_found += 1
    ind += 1
print(rhyme_pairs)

rhyme_words = [rhyme_pairs[0][0],
               rhyme_pairs[1][0],
               rhyme_pairs[0][1],
               rhyme_pairs[1][1],
               rhyme_pairs[2][0],
               rhyme_pairs[3][0],
               rhyme_pairs[2][1],
               rhyme_pairs[3][1],
               rhyme_pairs[4][0],
               rhyme_pairs[5][0],
               rhyme_pairs[4][1],
               rhyme_pairs[5][1],
               rhyme_pairs[6][0],
               rhyme_pairs[6][1],
              ]
print(rhyme_words)

[['face', 'pace'], ['toiled', 'uphold'], ['burned', 'determined'], ['morrow', 'sorrow'], ['remains', 'declines'], ['denote', 'boat'], ['fight', 'quite']]
['face', 'toiled', 'pace', 'uphold', 'burned', 'morrow', 'determined', 'sorrow', 'remains', 'denote', 'declines', 'boat', 'fight', 'quite']


### Generate reversed poem with starting words

In [47]:
# GENERATE SIMPLE SONNET NO RHYME SCHEME 
import random

all_emissions = list(range(emission_size))
all_states = list(range(latent_size))
emission_words = list(syllables_dict.keys())
max_syllab = 10
poem = []
poem_syllabs = []
emission_size = O.shape[1]
latent_size = O.shape[0]
for i in range(14):
    syllable_count = 0
    t = 0
    states = [0]*(max_syllab+10)
    lines = []
    line_syllabs = []

    
    while syllable_count < max_syllab:
        # initialize last word as the rhyme word
        if syllable_count == 0:
            word = rhyme_words[i]
            idx = np.argmax(list(syllables_dict.keys()) == word)
            potential_states = []
            potential_states_prob = []
            for i in range(latent_size):
                if (O[i][idx] > 0):
                    potential_states.append(i)
                    potential_states_prob.append(O[i][idx])
            states[t] = random.choices(potential_states, weights = potential_states_prob)
            idx = random.choices(all_emissions, weights = O[states[t]][0])[0]
            
        else:
            states[t] = random.choices(all_states, weights = A[states[t-1]][0])
            idx = random.choices(all_emissions, weights = O[states[t-1]][0])[0]
            word = emission_words[idx]
#             print(states, idx, word)
        
        # update syllable count
        word_syllables = syllables_dict[word]["syllabs"]
        syllab = random.choices(word_syllables, weights = [1/len(word_syllables)]*len(word_syllables))[0]
         
        # if exceeds syllable count, try again
        if (syllable_count + syllab) > max_syllab:
            while (syllable_count + syllab) > max_syllab:
                states[t] = random.choices(all_states, weights = A[states[t-1]][0])
                idx = random.choices(all_emissions, weights = O[states[t-1]][0])[0]
                word = emission_words[idx]
                word_syllables = syllables_dict[word]["syllabs"]
                syllab = random.choices(word_syllables, weights = [1/len(word_syllables)]*len(word_syllables))[0]
        syllable_count += random.choices(word_syllables, weights = [1/len(word_syllables)]*len(word_syllables))[0]
        lines.append(word)
        line_syllabs.append(syllab)
        t += 1
    poem.append(lines)
    poem_syllabs.append(line_syllabs)
    
sonnet = ""
for i in range(len(poem)):
    line = poem[i]
    for ind in range(len(poem[i])-1, -1, -1):
        sonnet += str(line[ind])+ " "
    sonnet += "\n"
    
print(sonnet)
# print(np.array(poem))
# print([len(poem[i]) for i in range(len(poem))])
# print([len(poem_syllabs[i]) for i in range(len(poem_syllabs))])
# print([sum(poem_syllabs[i]) for i in range(len(poem_syllabs))])
# print(np.array(poem_syllabs))

there in i beauty as , or store out face 
thee when heaven breath in my no frailties toiled 
nor refusest thy so shall of bring pace 
my place by chide by should you . of uphold 
white lame bones that use again , ah like burned 
a vex pity do among found morrow 
fire that my where , dimmed much determined 
the foiled march man mourn when ills my sorrow 
but heavy my grace as love so remains 
these those that can do content , stay denote 
you yet thou have , me my fire crushed declines 
they why live at doth none praised thy twice boat 
do thee i lords this for then to thine fight 
bid worst my , all time i to not should quite 

